In [ ]:
import os
import pandas as pd
import re

# Function to clean and process the .xls files
def process_xls_file(file_path, output_dir):
    # Check if the file has the .xls extension
    if file_path.endswith('.xls'):
        # Define the new path with .tsv extension
        new_file_path = file_path.replace('.xls', '.tsv')
        
        # Rename the file
        os.rename(file_path, new_file_path)
        print(f"File renamed to: {new_file_path}")
        
        # Cleaning the .tsv file
        output_file = os.path.join(output_dir, os.path.basename(new_file_path).replace('.tsv', '_cleaned.tsv'))
        
        # Define patterns to remove, using regex for the month and year variation
        unwanted_phrases = [
            r"^Notes:", 
            r"^These representative exchange rates,", 
            r"^\(1\) which are in terms of U.S. dollars per currency unit,", 
            r"^Representative Exchange Rates for Selected Currencies for [A-Za-z]+ \d{4}",  
            r"^Representative Exchange Rates for Selected Currencies for [A-Za-z]+ \d{4} Continued"
        ]
        
        # Compile the regex patterns for better performance
        compiled_patterns = [re.compile(pattern) for pattern in unwanted_phrases]
        
        # Open the input file and remove unwanted lines
        with open(new_file_path, "r") as file:
            lines = file.readlines()

        # Filter out lines that match any of the compiled regex patterns
        cleaned_lines = [line for line in lines if not any(pattern.match(line) for pattern in compiled_patterns)]

        # Write the cleaned data back to a new file
        with open(output_file, "w") as file:
            file.writelines(cleaned_lines)

        print(f"File cleaned and saved as {output_file}")

        # Further processing: reading, splitting, and transposing
        # Load the cleaned TSV file
        df = pd.read_csv(output_file, sep='\t')

        # Split the dataset into two parts based on the columns or rows
        split_index = df[df['Currency'] == 'Currency'].index[0]

        df1 = df.iloc[:split_index].copy()  # First dataset
        df2 = df.iloc[split_index + 1:].copy()  # Second dataset

        # Fix the column names of the second dataset (remove the extra header)
        df2.columns = df.iloc[split_index]

        # Drop any columns that are completely empty (i.e., all NaN values)
        df1.dropna(axis=1, how='all', inplace=True)
        df2.dropna(axis=1, how='all', inplace=True)

        # Merge the two datasets on the 'Currency' column
        merged_df = pd.merge(df1, df2, on='Currency', how='outer')

        # Save the merged dataset
        merged_output_file = os.path.join(output_dir, os.path.basename(new_file_path).replace('.tsv', '_merged.tsv'))
        merged_df.to_csv(merged_output_file, sep='\t', index=False)
        print(f"Datasets merged and saved to {merged_output_file}")

        # Transpose the dataset
        transposed_df = merged_df.set_index('Currency').T

        # Save the transposed dataset
        transposed_output_file = os.path.join(output_dir, os.path.basename(new_file_path).replace('.tsv', '_transposed.tsv'))
        transposed_df.to_csv(transposed_output_file, sep='\t')

        print(f"Dataset transposed and saved to {transposed_output_file}")

# Main function to process all .xls files in a folder
def process_all_xls_in_folder(input_folder, output_folder):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".xls"):
            file_path = os.path.join(input_folder, filename)
            process_xls_file(file_path, output_folder)

# Set input folder containing .xls files and output folder for .tsv files
input_folder = '/path/to/input/folder'  # Replace with your input folder path
output_folder = '/path/to/output/folder'  # Replace with your output folder path

# Run the automation
process_all_xls_in_folder(input_folder, output_folder)